## This is the MainCarRental Notebook that imports the classes for the Car Rental Program Project and prints the main menu that interacts with the Customer.

In [1]:
from CarRental import Rental, Customer, CustomerDataManagement

In [2]:
def Print_Main_Menu():  # Main Menu # 
    print('\n')
    print("---------------------------------- Welcome To Luxury Car Rentals --------------------------------")
    print("1. Display Available Cars Inventory")
    print("2. Rent Vehicle(s)")
    print("3. Return Vehicle(s)")
    print("4. Exit Menu")
    print("--------------------------------------------------------------------------------------------------")
    print('\n')
    return input("Please select an option from the menu above: ")

In [3]:
def main():
    
    # The main method will call the Print_Main_Menu() function that will display the main menu to the user. As long as the user does not select the exit option, the program will continue to run.
    # Due to design constraints, all data is held in memory and will be lost once the program is terminated.
    # Once the user selects the exit option, the program will terminate and all data will be lost
    
    
    # The following objects are created to store the rental inventory data, customer information, and customer object as soon as the main program is invoked and disappear once the program is terminated.
    rental_vehicle = Rental()       # This is the rental class object that will be used to store the rental inventory data and rental information as long as the program is running 
    customer_record_management = CustomerDataManagement() # This stores the customer management object which is used to store the customer information as long as the program is running
    customer = None # This will be used to store the customer object which will be used to access the customer information and rental information
    
    # This method is used to retrieve the customer record from the database, which in this case is a dictionary in memory in the CustomerDataManagement class. If the customer record is found, the customer object is created and returned. If the customer record is not found, the method will return None.
    def get_customer_record(customer_id):
        customer_record = customer_record_management.find_customer(customer_id) # This method is used to find the customer record in the database
        if customer_record is not None: # If the record exists, the customer object is created and populated with the customer information
            customer_name=customer_record[1]['first_name'] + " " + customer_record[1]['last_name']
            customer = Customer(customer_record[1]['first_name'], customer_record[1]['last_name'], customer_record[0], customer_record[1]['email'], customer_record[1]['phone']) # This creates the customer object
            print("Welcome back, " + customer_name + "!")
            return customer # returns the customer object
        else:
            return customer_record   # returns None if the customer record is not found         
   
    while True: # This loop will continue to run until the user selects the exit option
        
        choice = Print_Main_Menu()    
        
        # The following 'If/Else' block will execute the user's choice from the main menu. The user can select from options 1, 2, 3 or 4. If the user selects option 4, the program will terminate.
        if choice == '1': # First choice is to display the available cars inventory
            print('\n')
            print("Available Cars Inventory")
            print("-------------------------------")
            rental_vehicle.display_inventory()  # This Rental method is used to display the rental inventory   
            input("Press Enter to continue..." )
            print('\n')  
        elif choice == '2': # Second choice is to start the rental process ONLY if all eligibility requirements are met.
            print("Rent Vehicle(s)")
            print("-------------------------------")
            str = input("Do you have an account with us? (yes/no): ")
            # First eligibility requirement is to have an account in the system. If customer has an account the program will ask for the driver's license number to retrieve the customer record.
            if str=='yes':
                customer_id = input("Enter your Driver's License Number: ")
                customer = get_customer_record(customer_id) # This CustomerDataManagement method is used to retrieve the customer record from the database
                active_rental=customer_record_management.check_for_active_rentals(rental_vehicle, customer_id) # This CustomerDataManagement method is used to check if the customer has an active rental it returns True if the customer has an active rental and False if the customer does not have an active rental.
                print('\n')
                print("Active Rental: ", active_rental)
                print('\n')
                # In the following 'If/Else' block, the program checks if the customer record was found. If found it checks further that the customer does not have an active rental.
                # If an active rental is found, the program will not allow the customer to rent another vehicle until the current rental is returned.
                # If the customer record was not found, the program will tell user no account was found and continues to loop back to main menu.
                # IF the customer record is found AND the customer does not have outstanding rentals the program proceeds to allow the customer to rent a vehicle.
                if customer is not None: 
                    if not active_rental: # Second eligibility is for the user with an account to not have an active rental
                        input("Press Enter to continue..." )
                        # This is where the logic block for requesting a car is executed by the customer.car_request() method
                        customer.car_request(rental_vehicle) 
                        continue  
                    else:
                        print("It looks like you have an active rental. Please return the vehicle(s) before renting another one.")
                        input("Press Enter to continue..." )
                        continue 
                else:
                    print("Customer ID not found. Please try again.")
                    input("Press Enter to continue...")
                    continue 
                                      
            elif str=='no': # If the customer does not have an account, the program will ask for the customer's information to create an account.
                age_verification = input("Enter your age: ")
                if not age_verification.isdigit(): # This checks if the user input is a number
                    continue
                if int(age_verification) < 18: # Third elibility requirement is that the user must be 18 years or older to rent a vehicle
                    print("You must be 18 years or older to rent a vehicle.")
                    input("Press Enter to continue...")
                    continue
                #####################################################################################################################################################################################
                # The following block does not check to make sure that the user input is a valid for only characters in the name, valid  email address or phone number. This is a future enhancement.
                ####################################################################################################################################################################################
                fname = input("Enter your first name: ")
                lname = input("Enter your last name: ")                 
                driver_license = input("Enter your driver's license number: ") # This is the most important piece of information as it used as the key for all rentals and customer information
                email = input("Enter your email address: ")
                phone = input("Enter your phone number: ")
                duplicate_license = customer_record_management.checkDuplicate(driver_license) # This checks if the customer already exists in the database
                # If the customer already exists in the database, the program will not allow the customer to create a new account and will loop back to the main menu
                if duplicate_license:
                    print("Customer already exists in the database. Please try again.")
                    input("Press Enter to continue...")
                    continue
                else: # If no duplicate account is found, the system will create a new account with the User's input information and allow the customer to continue with the rental process, by calling the customer.car_request() method
                    customer_record_management.add_customer(fname, lname, driver_license, email, phone)
                    customer = Customer(fname, lname, driver_license, email, phone)
                    customer_record_management.display_customers(driver_license)
                    input("Press Enter to continue..." )
                    # This is where the logic block for requesting a car is executed by the customer.car_request() method
                    customer.car_request(rental_vehicle)
                    continue
        elif choice == '3': # Third choice is to return the vehicle(s) that the customer has rented
            print("Return Vehicle(s)")
            print("-------------------------------")
            dl=input("Enter your Driver's License Number: ")
            customer=get_customer_record(dl)
            if customer is not None: # A customer record must be found in order to proceed with the return process. If the customer record is not found, the program will loop back to the main menu.
                print("Found it!")
                # Next check to make sure that the customer has an active rental. If the customer does not have an active rental, the program will not allow the customer to try to return a vehicle.
                active_rental=customer_record_management.check_for_active_rentals(rental_vehicle, dl)
                if not active_rental:
                    print("It looks like you do not have an active rental. Please rent a vehicle first.")
                    input("Press Enter to continue...")
                    #continue
                else:   
                    customer.car_return(dl,rental_vehicle,customer_record_management) # This is where the logic block for returning a car is executed by the customer.car_return() method
            else: # If the customer record is not found, the program will loop back to the main menu
                print("Customer ID not found. Please try again.")
                input("Press Enter to continue...")
                #continue
            continue # This will loop back to the main menu
        elif choice == '4': # This last choice ends the program and all data is lost.
            print("Exit Menu")
            print("-------------------------------")
            print("Thank you for using the Car Rental System. Goodbye!")
            break
        else:
            print("Invalid Input. Please try again.")
            continue

    


In [ ]:
main()




---------------------------------- Welcome To Luxury Car Rentals --------------------------------
1. Display Available Cars Inventory
2. Rent Vehicle(s)
3. Return Vehicle(s)
4. Exit Menu
--------------------------------------------------------------------------------------------------




Please select an option from the menu above:  1




Available Cars Inventory
-------------------------------
Vehicle ID :Vin1 | Ferrari 488 Pista Coupe 2020 Color: Red Passengers: 2 Luggage: 4 Transmission: Auto Doors: 2 Rates: Hourly: $105 Daily: $2520 Weekly: $17640
Vehicle ID :Vin2 | Lamborghini Aventador SVJ 2022 Color: Black Passengers: 4 Luggage: 4 Transmission: Auto Doors: 4 Rates: Hourly: $125 Daily: $3000 Weekly: $21000
Vehicle ID :Vin3 | McLaren 720S 2022 Color: Blue Passengers: 2 Luggage: 2 Transmission: Auto Doors: 2 Rates: Hourly: $105 Daily: $2520 Weekly: $17640
Vehicle ID :Vin4 | Porsche 911 GT3 2022 Color: White Passengers: 2 Luggage: 2 Transmission: Auto Doors: 2 Rates: Hourly: $85 Daily: $2040 Weekly: $14280
Vehicle ID :Vin5 | Audi R8 2020 Color: Silver Passengers: 2 Luggage: 4 Transmission: Manual Doors: 2 Rates: Hourly: $125 Daily: $3000 Weekly: $21000
Vehicle ID :Vin6 | BMW M8 2022 Color: Black Passengers: 2 Luggage: 2 Transmission: Manual Doors: 2 Rates: Hourly: $115 Daily: $2760 Weekly: $19320
Vehicle ID :Vin7 |

Press Enter to continue... 






---------------------------------- Welcome To Luxury Car Rentals --------------------------------
1. Display Available Cars Inventory
2. Rent Vehicle(s)
3. Return Vehicle(s)
4. Exit Menu
--------------------------------------------------------------------------------------------------




...

Validated time: Sunday, April 21, 2024 08:00 AM


Rental Duration Details:
-------------------------
duration :  13:00:00
total_hours :  13.0
Rental Period: Hourly
-------------------------------------------------------------------------------
Rental Details:
-------------------------------------------------
Vehicle(s):
Vehicle: Porsche 911 Carrera 4S Cabriolet 2023 Color: Red Passengers: 2 Luggage: 2 Transmission: Auto Doors: 2 Rental Period: Hourly Rate: $80
-------------------------------------------------
Pickup Time:  Saturday, April 20, 2024 07:00 PM
Return Time:  Sunday, April 21, 2024 08:00 AM
Ontime Return Cost:  1040.0


NOTE: All vehicles must be rented at the same time and returned at the same time. Early returns will be charged the full rental rate.


LATE FEE INFORMATION : 
For Hourly Rentals: There will be a $20 late fee added to the hourly rate for every hour late.




Would you like to confirm this reservation? (Y/N):  y


Reservation confirmed. Thank you!


---------------------------------- Welcome To Luxury Car Rentals --------------------------------
1. Display Available Cars Inventory
2. Rent Vehicle(s)
3. Return Vehicle(s)
4. Exit Menu
--------------------------------------------------------------------------------------------------




Please select an option from the menu above:  1




Available Cars Inventory
-------------------------------
Vehicle ID :Vin1 | Ferrari 488 Pista Coupe 2020 Color: Red Passengers: 2 Luggage: 4 Transmission: Auto Doors: 2 Rates: Hourly: $105 Daily: $2520 Weekly: $17640
Vehicle ID :Vin2 | Lamborghini Aventador SVJ 2022 Color: Black Passengers: 4 Luggage: 4 Transmission: Auto Doors: 4 Rates: Hourly: $125 Daily: $3000 Weekly: $21000
Vehicle ID :Vin3 | McLaren 720S 2022 Color: Blue Passengers: 2 Luggage: 2 Transmission: Auto Doors: 2 Rates: Hourly: $105 Daily: $2520 Weekly: $17640
Vehicle ID :Vin4 | Porsche 911 GT3 2022 Color: White Passengers: 2 Luggage: 2 Transmission: Auto Doors: 2 Rates: Hourly: $85 Daily: $2040 Weekly: $14280
Vehicle ID :Vin5 | Audi R8 2020 Color: Silver Passengers: 2 Luggage: 4 Transmission: Manual Doors: 2 Rates: Hourly: $125 Daily: $3000 Weekly: $21000
Vehicle ID :Vin6 | BMW M8 2022 Color: Black Passengers: 2 Luggage: 2 Transmission: Manual Doors: 2 Rates: Hourly: $115 Daily: $2760 Weekly: $19320
Vehicle ID :Vin7 |